## A simple game

We need to define actions, states, state transitions, rewards and the discount factor.

An MDP is a 5-tuple, $\langle S, A, R, P, \gamma \rangle$

--16 states

--4 actions



In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)


# Define the gridworld environment
n_states = 16  # Total number of states in the grid (4x4 grid)
n_actions = 4  # Total number of possible actions: up, down, left, right
# Initialize the transition probabilities matrix
# It is a 3D array where P[s, a, s'] represents the probability of transitioning from state s to state s' given action a
P = np.zeros((n_states, n_actions, n_states))
# Initialize the rewards matrix
# R[s, a, s'] represents the reward received when transitioning from state s to state s' given action a
R = np.zeros((n_states, n_actions, n_states))
gamma = 0.9  # Discount factor for future rewards

# Fill in the transition probabilities and rewards for each state and action
for s in range(n_states):  # Loop over all states
    for a in range(n_actions):  # Loop over all actions
        if s == 0 or s == 15:  # If the current state is a terminal state (0 or 15)
            P[s, a, s] = 1  # Stay in the same state with probability 1
        else:  # If the current state is not terminal
            if a == 0:  # Up action
                s_prime = s - 4  # Calculate the new state after moving up
            elif a == 1:  # Down action
                s_prime = s + 4  # Calculate the new state after moving down
            elif a == 2:  # Left action
                s_prime = s - 1  # Calculate the new state after moving left
            else:  # Right action
                s_prime = s + 1  # Calculate the new state after moving right

            # Ensure the new state is within bounds
            if s_prime < 0:
                s_prime = 0  # If moving out of grid upwards, stay at the top
            if s_prime > 15:
                s_prime = 15  # If moving out of grid downwards, stay at the bottom

            # Assign rewards based on the new state
            if s_prime == 0:
                R[s, a, s_prime] = -1  # Penalty for moving to the start state
            elif s_prime == 15:
                R[s, a, s_prime] = 10  # Reward for reaching the goal state
            else:
                R[s, a, s_prime] = -1  # Penalty for non-terminal states to encourage reaching the goal

            P[s, a, s_prime] = 1  # Set the transition probability to 1 for the determined action

print(P)
# P[s, a, s_prime]


[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0

## Solution - DP

In [ ]:
# Define an arbitrary policy for now
# This policy is uniformly random, meaning at each state, each action is equally likely.
policy = np.ones((n_states, n_actions)) / n_actions
print(policy)

for s in range(n_states):
  print("(State ", s, ") Actions", policy[s])

# Initialize the value function to zeros for all states.
# V[s] represents the expected return starting from state s and following the given policy.
V = np.zeros(n_states)
print("Initial value function:")
print(V.reshape(4, 4))  # Reshape for better readability, assuming a 4x4 grid.

tolerance = 1e-6  # Convergence tolerance, stops the algorithm when changes are small.

# Policy Evaluation Loop
while True:
    delta = 0  # Initialize delta to track the maximum change in value function across all states in an iteration.
    for s in range(n_states):  # Iterate over all states.
        v = V[s]  # Store the current value of the state.
        bellman_update = 0  # This will accumulate the sum of expected returns.
        for a in range(n_actions):  # Iterate over all actions.
            for s_prime in range(n_states):  # Iterate over all possible next states.
                # Calculate the expected return using the Bellman equation for policy evaluation.
                bellman_update += policy[s, a] * P[s, a, s_prime] * (R[s, a, s_prime] + gamma * V[s_prime])
        V[s] = bellman_update  # Update the value function with the computed expected return.
        delta = max(delta, abs(v - V[s]))  # Update delta with the largest change in value function for any state.
    if delta < tolerance:  # If the maximum change is less than the tolerance, the value function has converged.
        break

print("Final value function:")
print(V.reshape(4, 4))  # Print the final value function, reshaped into the 4x4 grid.

[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
(State  0 ) Actions [0.25 0.25 0.25 0.25]
(State  1 ) Actions [0.25 0.25 0.25 0.25]
(State  2 ) Actions [0.25 0.25 0.25 0.25]
(State  3 ) Actions [0.25 0.25 0.25 0.25]
(State  4 ) Actions [0.25 0.25 0.25 0.25]
(State  5 ) Actions [0.25 0.25 0.25 0.25]
(State  6 ) Actions [0.25 0.25 0.25 0.25]
(State  7 ) Actions [0.25 0.25 0.25 0.25]
(State  8 ) Actions [0.25 0.25 0.25 0.25]
(State  9 ) Actions [0.25 0.25 0.25 0.25]
(State  10 ) Actions [0.25 0.25 0.25 0.25]
(State  11 ) Actions [0.25 0.25 0.25 0.25]
(State  12 ) Actions [0.25 0.25 0.25 0.25]
(State  13 ) Actions [0.25 0.25 0.25 0.25]
(State  14 ) Actions [0.25 0.25 0.25 0.

## Q-learning algorithm

## Q-Learning Overview

Q-Learning is a model-free reinforcement learning algorithm used to inform an agent on how to act optimally in a given environment by learning the value of actions in states. It does not require a model of the environment (hence "model-free"), and it can handle problems with stochastic transitions and rewards, without needing adaptations.

### Key Concepts

- **Q-Value (Action-Value):** Represents the value of taking a particular action in a particular state, given that the agent will follow the optimal policy thereafter. The Q-value for a state-action pair \((s, a)\), denoted as \(Q(s, a)\), is the expected return (total discounted future reward) starting from state \(s\), taking action \(a\), and thereafter following an optimal policy.

- **Policy:** A strategy that the agent employs to determine the next action based on the current state. The optimal policy \(\pi^*\) maximizes the expected return from all states.

- **Reward:** A signal received from the environment in response to an action, indicating the immediate benefit of that action.

- **State-Action Pairs:** The combination of a state and an action within that state, often represented as \((s, a)\).

### The Q-Learning Algorithm

The Q-learning algorithm updates Q-values using the Bellman equation as follows:

$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'}Q(s', a') - Q(s, a) \right]
$$


where:
- $s$ is the current state,
- $a$ is the current action,
- $s'$ is the new state after taking action $a$,
- $r$ is the reward received after taking action $a$ in state $s$,
- $\alpha$ is the learning rate,
- $\gamma$ is the discount factor, and
- $\max_{a'}Q(s', a')$ is the maximum predicted Q-value for the next state $s'$, representing the best future reward that can be obtained.



In [ ]:
# Initialize the Q-values matrix with zeros for all state-action pairs.
# Q[s, a] represents the value of taking action a in state s.
print(P)
Q = np.zeros((n_states, n_actions))
print("Q", Q)
n_episodes = 10000  # Number of episodes for the agent to learn from.
alpha = 0.1  # Learning rate, determining how much new information overrides old information.
epsilon = 0.1  # Epsilon-greedy exploration probability
# Loop over each episode.
for episode in range(n_episodes):
    # Randomly select an initial state for the beginning of the episode.
    s = np.random.randint(n_states)
    # Continue the episode until reaching a terminal state (0 or 15 in this gridworld).
    while s not in [0, 15]:
        # Epsilon-greedy action selection:
        # With probability epsilon, select a random action (exploration).
        if np.random.uniform() < epsilon:
            a = np.random.randint(n_actions)
        else:
            # With probability 1 - epsilon, select the action with the highest Q-value.
            a = np.argmax(Q[s, :])
        # Take the selected action, observe the next state and reward.
        # The next state is chosen based on the transition probabilities P[s, a, :].
        s_prime = np.random.choice(range(n_states), p=P[s, a, :])
        r = R[s, a, s_prime]  # The reward received for the (s, a, s') transition.
        # Update the Q-value for the current state-action pair using the Q-learning formula.
        # This incorporates the immediate reward and the discounted value of the best future action.
        Q[s, a] = Q[s, a] + alpha * (r + gamma * np.max(Q[s_prime, :]) - Q[s, a])
        # Move to the next state.
        s = s_prime

# After learning, extract the optimal policy from the Q-values.
# The optimal action for each state is the one with the highest Q-value.
policy = np.argmax(Q, axis=1)
# print("updated Q: ", Q)
print("Optimal policy:")
# Print the optimal policy, reshaped into the grid format for easy visualization.
print(policy.reshape(4, 4))

[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0